In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { load } from "dotenv";

const env = await load();
const process = {
  env,
};

// 模拟调用出错的 chain
const errorLLM = new ChatOpenAI({
  openAIApiKey: '123', // ✅ 顶层参数
  configuration: {
    baseURL: "https://api.deepseek.com/v1",
  },
  modelName: "deepseek-chat",
  // 大多 runnable 都自带出错重试的机制，所以将重试的次数 maxRetries 设置为 0，避免重复重试
  maxRetries: 0,
});

// 兜底模型 chain
const fallbackLLM = new ChatOpenAI({
  openAIApiKey: process.env.DEEPSEEK_API_KEY, // ✅ 顶层参数
  configuration: {
    baseURL: "https://api.deepseek.com/v1",
  },
  modelName: "deepseek-chat",
});

// 创建带有兜底模型的 chain
// - errorLLM 作为主 chain
// - fallbackLLM 作为兜底 chain
const llmWithFallback = errorLLM.withFallbacks({
    fallbacks: [fallbackLLM],
});

// await errorLLM.invoke("你好");
await llmWithFallback.invoke("你好");